In [5]:
# from dotenv import load_dotenv
# import os

# load_dotenv()

# API_KEY = os.getenv("API_KEY")

In [6]:
import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"])

In [71]:
# prompt = "what are the contents of sajith premadasa's manifesto"
prompt = "how will sajith premadasa control the expenditure"

In [72]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)
print(response.text)

I cannot provide you with an opinion or prediction on how Sajith Premadasa will control government expenditure. This is because:

* **I am an AI:** I do not have access to real-time information, political insights, or the ability to predict future events. 
* **Political Issues are Complex:** Controlling government expenditure is a complex issue influenced by various factors including economic conditions, political priorities, and public opinion. 

To understand potential approaches, I recommend researching the following:

* **Sajith Premadasa's economic policies:** Look for public statements, manifestos, or interviews where he outlines his vision for managing the economy. 
* **Sri Lankan economic challenges:**  Investigate the specific economic problems Sri Lanka faces, such as debt, inflation, or unemployment. This will provide context for his proposed solutions.
* **Expert opinions:** Seek out analysis from economists, political analysts, and financial institutions who study Sri Lank

In [40]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Sajith_manifesto_english.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
# data

In [41]:
len(data) # 44 pages

44

In [42]:
data[7] # first page

Document(metadata={'source': 'Sajith_manifesto_english.pdf', 'page': 7}, page_content='8\nBUILD A \nRESILIENT \nECONOMY\nWe prioritize equitable growth \nthrough a Social Market Economy \nmodel, ensuring that the benefits of \ndevelopment reach all citizens. As \nwe chart a course toward economic \nrecovery and lasting prosperity, we \npledge transparency, sustainability \nand empowerment creating a future \nwhere all can thrive.\n')

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  281


In [46]:
docs[80] # first document

Document(metadata={'source': 'Sajith_manifesto_english.pdf', 'page': 14}, page_content='participation in developing the seaports.\nFISHERIES AND \nAQUATIC RESOURCES\nLeverage Sri Lanka’s maritime potential by \ndeveloping blue economy initiatives that \npromote sustainable economic activities \nthrough public-private partnerships (PPP), \nmaximizing the utilization of aquatic \nresources of the Sea of Sri Lanka.\n• Establishing a low-interest loan scheme \nthrough state and private banks \nfor entrepreneurs to economically \nstrengthen the fishing community and')

In [51]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, sajeev!")
vector[:5]
#vector

[0.023708494380116463,
 -0.0038761459290981293,
 -0.0130457216873765,
 -0.0558636412024498,
 0.05156157165765762]

In [52]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [74]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke(prompt)

In [75]:
len(retrieved_docs)

10

In [76]:
retrieved_docs

[Document(metadata={'page': 9, 'source': 'Sajith_manifesto_english.pdf'}, page_content='10\nMONETARY AND  \nEXCHANGE RATE \nPOLICY\nWe will respect the autonomy of the Central \nBank of Sri Lanka (CBSL) provided by the \nnew Act but hold the bank fully accountable \nfor meeting its inflation target. Furthermore, \nwe will help ensure low and stable interest \nrates, an internationally competitive foreign \nexchange rate..\nACHIEVING A \nREVENUE GROWTH\nThe solution to increase government \nrevenue is not confined to raising taxes. One \nof our first acts will be to introduce digital'),
 Document(metadata={'page': 8, 'source': 'Sajith_manifesto_english.pdf'}, page_content='government tax and expenditure proposals.\nMANAGING THE DEBT \nCRISIS AND THE IMF\nWe will strengthen fiscal discipline as per the \nIMF agreement and continue with difficult \nbut necessary reforms to meet the targets \nof revenue, primary balance and debt. \nHowever, we will make amendments to \nthe current programm

In [77]:
print(retrieved_docs[5].page_content)

competition law.
We will bring in transparency in public 
finance management. There shall be full 
disclosure on expenditure plans and debt 
management strategies as well as in tax 
administration. The establishment of 
the Parliamentary Budget Office would 
henceforth provide detailed analysis of all 
government tax and expenditure proposals.
MANAGING THE DEBT 
CRISIS AND THE IMF
We will strengthen fiscal discipline as per the 
IMF agreement and continue with difficult


In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=500)

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [62]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [73]:
response = rag_chain.invoke({"input": prompt})
print(response["answer"])

Sajith Premadasa's plan to control government expenditure includes:

* **Controlling wasteful spending and corruption:** This involves reforming state enterprises and using data-based planning to increase transparency and accountability.
* **Strengthening fiscal discipline:** This will be done in accordance with the IMF agreement.
* **Utilizing funds saved from expenditure control:** These funds will be used to implement a social safety net for the distressed population and to strengthen education and health sectors. 
* **Transparency in public finance management:** This includes full disclosure on expenditure plans, debt management strategies, and tax administration. 
* **Establishment of the Parliamentary Budget Office:** This office will provide detailed analysis of all government tax and expenditure proposals. 

